### 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import leveldb
db60 = leveldb.LevelDB('data/db60')

In [2]:
import gzip
import json

In [3]:
for line in gzip.open('data/artist.json.gz'):
    data  = json.loads(line)
    name = (data['name'] + ' ' + str(data['id'])).encode()
    area = data.get('area', 'NODATA').encode()
    db60.Put(name, area)

### 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [4]:
def where(name):
    for k, v in db60.RangeIter(key_from=name.encode(), key_to=(name + chr(33)).encode()):
        artist = k.decode().split()[0]
        place = v.decode()
        print(artist + '\t' + place)

In [5]:
where('Oasis')

Oasis	United Kingdom
Oasis	United States
Oasis	United Kingdom
Oasis	NODATA
Oasis	NODATA
Oasis	Kent


### 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [6]:
i = 0
for k, v in db60.RangeIter():
    if v.decode() == 'Japan':
        i = i + 1
print(i)

22821


### 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [7]:
db63 = leveldb.LevelDB('data/db63')

In [8]:
for line in gzip.open('data/artist.json.gz'):
    data  = json.loads(line)
    name = (data['name'] + ' ' + str(data['id'])).encode()
    if 'tags' in data:
        tags = ' '.join([tag['value'] + ' ' + str(tag['count']) for tag in data['tags']]).encode()
    else:
        tags = 'NOTAGS'.encode()
    db63.Put(name, tags)

In [9]:
def taged(name):
    for k, v in db63.RangeIter(key_from=name.encode(), key_to=(name + chr(33)).encode()):
        artist = k.decode().split()[0]
        tags = v.decode()
        print(artist + '\t' + tags)

In [10]:
taged('Oasis')

Oasis	rock 1 britpop 3 british 4 uk 1 britannique 1 rock and indie 1 england 1 manchester 1
Oasis	NOTAGS
Oasis	morning glory 1 oasis 1
Oasis	NOTAGS
Oasis	NOTAGS
Oasis	NOTAGS


### 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value